In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
from __future__ import division
from sympy import *
import numpy as np

init_printing(fontsize='15pt',use_latex='mathjax')

###  Gives the generalized (associated) Laguerre polynomial


The Laguerre polynomials are orthogonal with respect to the weight $z^a e^{−z}$ on [0,∞)[0,∞).



###  orthogonality

###  $R_l(\tau)$

In [2]:
tau, tauprime, taub = symbols("tau tauprime tau_b", positive=True)
b, cn, Ln = symbols("b, c_n L_n")
cnprime, Lnprime = symbols("c_nprime L_nprime")
Ln, Rl = symbols("L_n, R_l", cls=Function)
Lnprime = symbols("L_nprime", cls=Function)
x, a, b, omegaxi = symbols("x a b omega_xi", positive=True)
n, nprime, k, l, lprime, n0 = symbols("n nprime k l lprime n0", integer=True)


Rl=Sum((tau/taub)**abs(l)*exp(-b*tau**2)*cnprime**abs(l)*Lnprime(a* tau**2)**abs(l), (nprime,0,oo))
Rlprime_tauprime = Rl.subs(tau,tauprime).subs(l,lprime)
Rlprime_tauprime

  ∞                                              
 ____                                            
 ╲                                               
  ╲                  │l′│                       2
   ╲       │l′│ ⎛ τ′⎞         │l′│⎛    2⎞  -b⋅τ′ 
   ╱   c_n′    ⋅⎜───⎟    ⋅L_n′    ⎝a⋅τ′ ⎠⋅ℯ      
  ╱             ⎝τ_b⎠                            
 ╱                                               
 ‾‾‾‾                                            
n′ = 0                                           

##  $g_0(\tau)$

In [3]:
Lk0 = symbols('L_k^0',cls=Function)
gk = symbols('g_k')
g0=exp(-b*tau**2)*Sum(gk*Lk0(a* tau**2), (k,0,n0))
g0

         n₀                  
        ___                  
     2  ╲                    
 -b⋅τ    ╲             ⎛   2⎞
ℯ     ⋅  ╱   g_k⋅L_k__0⎝a⋅τ ⎠
        ╱                    
        ‾‾‾                  
       k = 0                 

## 1) RHS Damper part

### 1.1) Expression from DELPHI-expanded.pdf

Expression as in DELPHI-expanded, slide 34 (without $\frac{\mu}{\omega_0} \kappa g_0$)

In [4]:
Jl, Jlprime = symbols("J_l J_lprime",cls=Function)

expr=Integral(I**(lprime-l) * tauprime * Rlprime_tauprime* Jlprime(-omegaxi* tauprime)* Jl(-omegaxi*tau),
 (tauprime, 0, oo))

expr=Sum(expr,(lprime, -oo, oo))
expr=simplify(expand_power_base(expr))

expr

    ∞                                                                         
_________                                                                     
╲                                                                             
 ╲        ∞                                                                   
  ╲       ⌠                                                                   
   ╲      ⎮                                         ∞                         
    ╲     ⎮                                        ____                       
     ╲    ⎮                                        ╲                          
      ╲   ⎮                                         ╲                         
       ╲  ⎮  -l + l′                                 ╲       │l′│    -│l′│   │
       ╱  ⎮ ⅈ       ⋅τ′⋅J_l(-ωₓᵢ⋅τ)⋅J_l′(-ωₓᵢ⋅τ′)⋅   ╱   c_n′    ⋅τ_b     ⋅τ′ 
      ╱   ⎮                                         ╱                         
     ╱    ⎮                                        ╱

!!! The sums over $l'$ and $n'$ are not written in front !!!

In [5]:
newexpr=expr.args[0].args[0].args[0]*\
expr.args[0].args[0].args[1]*\
expr.args[0].args[0].args[2]*\
expr.args[0].args[0].args[3]*\
expr.args[0].args[0].args[4].args[0]

newexpr

                                                                              
 -l + l′     │l′│    -│l′│      │l′│                               │l′│⎛    2⎞
ⅈ       ⋅c_n′    ⋅τ_b     ⋅τ′⋅τ′    ⋅J_l(-ωₓᵢ⋅τ)⋅J_l′(-ωₓᵢ⋅τ′)⋅L_n′    ⎝a⋅τ′ ⎠

       2
  -b⋅τ′ 
⋅ℯ      

In [6]:
fact = np.prod([newexpr.args[ii] for ii in [1,2,3,6]])

In [7]:
newexpr=fact*simplify(Integral(newexpr/fact,(tauprime,0,oo)))

newexpr

                                       ∞                                      
                                       ⌠                                      
                                       ⎮                                      
 -l + l′     │l′│    -│l′│             ⎮   │l′│ + 1                   │l′│⎛   
ⅈ       ⋅c_n′    ⋅τ_b     ⋅J_l(-ωₓᵢ⋅τ)⋅⎮ τ′        ⋅J_l′(-ωₓᵢ⋅τ′)⋅L_n′    ⎝a⋅τ
                                       ⌡                                      
                                       0                                      

                 
                 
          2      
 2⎞  -b⋅τ′       
′ ⎠⋅ℯ       d(τ′)
                 
                 

### 1.2) Laguerre integral from Chao

Integral as from Chao p.316, table 6.3, formula 3

In [8]:
eta, alpha, y = symbols('eta, alpha, y')

laguerre_lhs=Integral(x**(lprime+1) *exp(-eta*x**2) *Jlprime(x*y)* Lnprime(alpha * x**2)**lprime,(x, 0, oo))

laguerre_lhs

∞                                         
⌠                                         
⎮                                     2   
⎮  l′ + 1               l′⎛   2⎞  -η⋅x    
⎮ x      ⋅J_l′(x⋅y)⋅L_n′  ⎝α⋅x ⎠⋅ℯ      dx
⌡                                         
0                                         

In [9]:
temp=laguerre_lhs.subs(alpha,a).subs(lprime,abs(lprime)).subs(y,-omegaxi).subs(eta,b)
laguerre1_lhs=temp.transform(x,tauprime)

laguerre1_lhs

∞                                                       
⌠                                                       
⎮                                                2      
⎮   │l′│ + 1                   │l′│⎛    2⎞  -b⋅τ′       
⎮ τ′        ⋅J_l′(-ωₓᵢ⋅τ′)⋅L_n′    ⎝a⋅τ′ ⎠⋅ℯ       d(τ′)
⌡                                                       
0                                                       

Solution

In [10]:
laguerre_rhs=(eta- alpha)**nprime/(2**(lprime+1)*eta**(lprime+nprime+1))*y**lprime*exp(-y**2/(4*eta))*\
Lnprime(alpha*y**2/(4*eta*(alpha-eta)))**lprime
laguerre_rhs

                                                             2 
                                                           -y  
                                            ⎛       2   ⎞  ────
 -l′ - 1  -l′ - n′ - 1  l′         n′     l′⎜    α⋅y    ⎟  4⋅η 
2       ⋅η            ⋅y  ⋅(-α + η)  ⋅L_n′  ⎜───────────⎟⋅ℯ    
                                            ⎝4⋅η⋅(α - η)⎠      

In [11]:
laguerre1_rhs=laguerre_rhs.subs(alpha,a).subs(lprime,abs(lprime)).subs(y,-omegaxi).subs(eta,b)

laguerre1_rhs

                                                                            2 
                                                                        -ωₓᵢ  
                                                         ⎛        2  ⎞  ──────
 -│l′│ - 1  -n′ - │l′│ - 1       │l′│         n′     │l′│⎜   a⋅ωₓᵢ   ⎟   4⋅b  
2         ⋅b              ⋅(-ωₓᵢ)    ⋅(-a + b)  ⋅L_n′    ⎜───────────⎟⋅ℯ      
                                                         ⎝4⋅b⋅(a - b)⎠        

### 1.3)  Substituting Laguerre 1 in DELPHI's expression

In [12]:
newexpr0=Sum(simplify(newexpr.subs(laguerre1_lhs,laguerre1_rhs)),(lprime, -oo, oo),(nprime, 0, oo))
newexpr0

  ∞       ∞                                                                   
______  ______                                                                
╲       ╲                                                                     
 ╲       ╲                                                                    
  ╲       ╲                                        ⎛              ⎛        2  
   ╲       ╲                                       ⎜              ⎜   a⋅ωₓᵢ   
    ╲       ╲                                      ⎜-c_n′⋅ωₓᵢ⋅L_n′⎜───────────
    ╱       ╱   -│l′│ - 1  -l + l′  -n′ - │l′│ - 1 ⎜              ⎝4⋅b⋅(a - b)
   ╱       ╱   2         ⋅ⅈ       ⋅b              ⋅⎜──────────────────────────
  ╱       ╱                                        ⎝            τ_b           
 ╱       ╱                                                                    
╱       ╱                                                                     
‾‾‾‾‾‾  ‾‾‾‾‾‾                                      

In [13]:
np.prod([newexpr0.args[0].args[ii] for ii in [0,1,2,4,5,6]])

                                                                2 
                                                            -ωₓᵢ  
                                                            ──────
 -│l′│ - 1  -l + l′  -n′ - │l′│ - 1         n′               4⋅b  
2         ⋅ⅈ       ⋅b              ⋅(-a + b)  ⋅J_l(-ωₓᵢ⋅τ)⋅ℯ      

In [14]:
(np.prod([newexpr0.args[0].args[3].args[0].args[ii] for ii in [0,1,2,3]]))**\
(newexpr0.args[0].args[3].args[1])*\
(newexpr0.args[0].args[3].args[0].args[4])**(newexpr0.args[0].args[3].args[1])

            │l′│         ⎛        2  ⎞
⎛-c_n′⋅ωₓᵢ ⎞         │l′│⎜   a⋅ωₓᵢ   ⎟
⎜──────────⎟    ⋅L_n′    ⎜───────────⎟
⎝   τ_b    ⎠             ⎝4⋅b⋅(a - b)⎠

In [15]:
newexpr0=np.prod([newexpr0.args[0].args[ii] for ii in [0,1,2,4,5,6]])*\
(np.prod([newexpr0.args[0].args[3].args[0].args[ii] for ii in [0,1,2,3]]))**\
(newexpr0.args[0].args[3].args[1])*\
(newexpr0.args[0].args[3].args[0].args[4])**(newexpr0.args[0].args[3].args[1])
newexpr0

                                                                              
                                                                              
                                                │l′│                          
 -│l′│ - 1  -l + l′  -n′ - │l′│ - 1 ⎛-c_n′⋅ωₓᵢ ⎞             n′               
2         ⋅ⅈ       ⋅b              ⋅⎜──────────⎟    ⋅(-a + b)  ⋅J_l(-ωₓᵢ⋅τ)⋅L_
                                    ⎝   τ_b    ⎠                              

                         2 
                     -ωₓᵢ  
      ⎛        2  ⎞  ──────
  │l′│⎜   a⋅ωₓᵢ   ⎟   4⋅b  
n′    ⎜───────────⎟⋅ℯ      
      ⎝4⋅b⋅(a - b)⎠        

In [16]:
newexpr0 = Sum(newexpr0, (lprime, -oo, oo), (nprime, 0, oo))
newexpr0

  ∞       ∞                                                                   
_____   _____                                                                 
╲       ╲                                                                     
 ╲       ╲                                                                    
  ╲       ╲                                                                   
   ╲       ╲                                                   │l′│           
   ╱       ╱    -│l′│ - 1  -l + l′  -n′ - │l′│ - 1 ⎛-c_n′⋅ωₓᵢ ⎞             n′
  ╱       ╱    2         ⋅ⅈ       ⋅b              ⋅⎜──────────⎟    ⋅(-a + b)  
 ╱       ╱                                         ⎝   τ_b    ⎠               
╱       ╱                                                                     
‾‾‾‾‾   ‾‾‾‾‾                                                                 
n′ = 0 l′ = -∞                                                                

                                          
        

### 1.4) Product with $\frac{\mu}{\omega_0} k g_0$

In [17]:
kappa = symbols('kappa')
mu, omega0 = symbols("mu omega_0")

newexpr2 = mu/omega0 * kappa * g0* newexpr0

newexpr2

                                      ∞       ∞                               
                                    _____   _____                             
           ⎛  n₀                  ⎞ ╲       ╲                                 
           ⎜ ___                  ⎟  ╲       ╲                                
         2 ⎜ ╲                    ⎟   ╲       ╲                               
     -b⋅τ  ⎜  ╲             ⎛   2⎞⎟    ╲       ╲                              
κ⋅μ⋅ℯ     ⋅⎜  ╱   g_k⋅L_k__0⎝a⋅τ ⎠⎟⋅   ╱       ╱    -│l′│ - 1  -l + l′  -n′ - 
           ⎜ ╱                    ⎟   ╱       ╱    2         ⋅ⅈ       ⋅b      
           ⎜ ‾‾‾                  ⎟  ╱       ╱                                
           ⎝k = 0                 ⎠ ╱       ╱                                 
                                    ‾‾‾‾‾   ‾‾‾‾‾                             
                                    n′ = 0 l′ = -∞                            
────────────────────────────────────────────────────

In [18]:
newexpr3 = \
newexpr2.args[0]*\
newexpr2.args[1]*\
newexpr2.args[2]*\
newexpr2.args[3].args[0]*\
newexpr2.args[4].args[0]*\
newexpr2.args[5]

The product, once the sums have been put in front

!!! The sums are not written !!! They are on $0 \leq n' \leq \infty$

$-\infty \leq l' \leq \infty$

$0 \leq k \leq n_0$

In [19]:
newexpr3

                                                                              
                                                                              
                                                        │l′│                  
 -│l′│ - 1  -l + l′  -n′ - │l′│ - 1         ⎛-c_n′⋅ωₓᵢ ⎞             n′       
2         ⋅ⅈ       ⋅b              ⋅g_k⋅κ⋅μ⋅⎜──────────⎟    ⋅(-a + b)  ⋅J_l(-ω
                                            ⎝   τ_b    ⎠                      
──────────────────────────────────────────────────────────────────────────────
                                                                  ω₀          

                                              2        
                                          -ωₓᵢ         
                           ⎛        2  ⎞  ──────      2
            ⎛   2⎞     │l′│⎜   a⋅ωₓᵢ   ⎟   4⋅b    -b⋅τ 
ₓᵢ⋅τ)⋅L_k__0⎝a⋅τ ⎠⋅L_n′    ⎜───────────⎟⋅ℯ      ⋅ℯ     
                           ⎝4⋅b⋅(a - b)⎠               
───────────────────────────────

### 1.5) Multiplying by the weight function and integrating in $\tau$

The weight function is necessary for the orthogonality on the LHS

In [37]:
qy0, omega0, omegas = symbols('Q_y_0 omega_0 omega_s')

weight = Ln(a*tau**2)**abs(l)*exp((-a+b)*tau**2)*tau**abs(l)*2*a*tau*a**(abs(l))
weight

                               2         
     │l│    │l│    │l│⎛   2⎞  τ ⋅(-a + b)
2⋅a⋅a   ⋅τ⋅τ   ⋅L_n   ⎝a⋅τ ⎠⋅ℯ           

In [21]:
rhs_out = newexpr3*weight
rhs_out

                                                                              
                                                                              
                                                                        │l′│  
   -│l′│ - 1  -l + l′    │l│  -n′ - │l′│ - 1            │l│ ⎛-c_n′⋅ωₓᵢ ⎞      
2⋅2         ⋅ⅈ       ⋅a⋅a   ⋅b              ⋅g_k⋅κ⋅μ⋅τ⋅τ   ⋅⎜──────────⎟    ⋅(
                                                            ⎝   τ_b    ⎠      
──────────────────────────────────────────────────────────────────────────────
                                                                              

                                                                           2  
                                                                       -ωₓᵢ   
                                                        ⎛        2  ⎞  ────── 
       n′                   ⎛   2⎞    │l│⎛   2⎞     │l′│⎜   a⋅ωₓᵢ   ⎟   4⋅b   
-a + b)  ⋅J_l(-ωₓᵢ⋅τ)⋅L_k__0⎝a⋅τ ⎠⋅L_n   ⎝a⋅τ ⎠⋅L_n

In [22]:
args=np.array(rhs_out.args)

rhs_fact=np.prod([args[ii] for ii in [0,1,2,3,4,6,7,8,9,10,12,13,15,20]])

rhs_fact

                                                                              
                                                                              
                                                                 │l′│         
   -│l′│ - 1  -l + l′    │l│  -n′ - │l′│ - 1         ⎛-c_n′⋅ωₓᵢ ⎞             
2⋅2         ⋅ⅈ       ⋅a⋅a   ⋅b              ⋅g_k⋅κ⋅μ⋅⎜──────────⎟    ⋅(-a + b)
                                                     ⎝   τ_b    ⎠             
──────────────────────────────────────────────────────────────────────────────
                                                      ω₀                      

                              2 
                          -ωₓᵢ  
           ⎛        2  ⎞  ──────
n′     │l′│⎜   a⋅ωₓᵢ   ⎟   4⋅b  
  ⋅L_n′    ⎜───────────⎟⋅ℯ      
           ⎝4⋅b⋅(a - b)⎠        
────────────────────────────────
                                

And integrating in $\tau$

In [23]:
#rhs_fact = 2*2**(-Abs(lprime) - 1)*I**(-l + lprime)*a*a**Abs(l)*b**(-nprime - Abs(lprime) - 1)*\
#gk*kappa*(-cnprime*omegaxi/taub)**Abs(lprime)*(-a + b)**nprime*\
#Lnprime(a*omegaxi**2/(4*b*(a - b)))**Abs(lprime)*exp(-omegaxi**2/(4*b))

rhs_int = np.prod([args[ii] for ii in [5,11,14,16,17,18,19]])
rhs_int = simplify(rhs_int)
rhs_int = Integral(rhs_int,(tau,0,oo))

rhs_int

∞                                                         
⌠                                                         
⎮                                                     2   
⎮  │l│ + 1                   ⎛   2⎞    │l│⎛   2⎞  -a⋅τ    
⎮ τ       ⋅J_l(-ωₓᵢ⋅τ)⋅L_k__0⎝a⋅τ ⎠⋅L_n   ⎝a⋅τ ⎠⋅ℯ      dτ
⌡                                                         
0                                                         

In [24]:
rhs_fact*rhs_int

                                                                              
                                                                              
                                                                 │l′│         
   -│l′│ - 1  -l + l′    │l│  -n′ - │l′│ - 1         ⎛-c_n′⋅ωₓᵢ ⎞             
2⋅2         ⋅ⅈ       ⋅a⋅a   ⋅b              ⋅g_k⋅κ⋅μ⋅⎜──────────⎟    ⋅(-a + b)
                                                     ⎝   τ_b    ⎠             
                                                                              
──────────────────────────────────────────────────────────────────────────────
                                                                              

                              2  ∞                                            
                          -ωₓᵢ   ⌠                                            
           ⎛        2  ⎞  ────── ⎮                                            
n′     │l′│⎜   a⋅ωₓᵢ   ⎟   4⋅b   ⎮  │l│ + 1        

### 1.6) Formula form Erdelyi, vol 2, p43, formula 8 

In [25]:
x, y, alpha, nu, sigma = symbols("x y alpha nu sigma")
i, j = symbols("i j")
Li, Lj = symbols("L_i L_j", cls=Function)
Jnu = symbols("J_nu", cls=Function)

In [26]:
erdel_int = Integral(x**(nu+S(1)/2)*exp(-alpha*x**2)*Li(alpha*x**2)**(nu-sigma)*\
                      Lj(alpha*x**2)**sigma*Jnu(x*y)*(x*y)**(S(1)/2),(x,0,oo))

erdel_int
simplify(erdel_int)

∞                                                             
⌠                                                             
⎮                                                         2   
⎮  ν + 1/2   _____            ν - σ⎛   2⎞    σ⎛   2⎞  -α⋅x    
⎮ x       ⋅╲╱ x⋅y ⋅J_ν(x⋅y)⋅Lᵢ     ⎝α⋅x ⎠⋅L_j ⎝α⋅x ⎠⋅ℯ      dx
⌡                                                             
0                                                             

In [27]:
erdel_sol = (-1)**(i+j)*(2*alpha)**(-nu-1)*y**(nu+S(1)/2)*exp(-y**2/(4*alpha))*\
Lj(y**2/(4*alpha))**(sigma-i+j)*Li(y**2/(4*alpha))**(nu-sigma+i-j)

erdel_sol

                                                                          2 
                                                                        -y  
                                              ⎛  2⎞              ⎛  2⎞  ────
    i + j  ν + 1/2      -ν - 1   i - j + ν - σ⎜ y ⎟    -i + j + σ⎜ y ⎟  4⋅α 
(-1)     ⋅y       ⋅(2⋅α)      ⋅Lᵢ             ⎜───⎟⋅L_j          ⎜───⎟⋅ℯ    
                                              ⎝4⋅α⎠              ⎝4⋅α⎠      

### 1.7) Using the formula

In [28]:
rhs_int

∞                                                         
⌠                                                         
⎮                                                     2   
⎮  │l│ + 1                   ⎛   2⎞    │l│⎛   2⎞  -a⋅τ    
⎮ τ       ⋅J_l(-ωₓᵢ⋅τ)⋅L_k__0⎝a⋅τ ⎠⋅L_n   ⎝a⋅τ ⎠⋅ℯ      dτ
⌡                                                         
0                                                         

- sigma is 0
- nu is abs(l)
- i is n
- j is k
- y is -omegaxi
- alpha is a

In [29]:
rhs_simpl = (-omegaxi)**(-S(1)/2)*erdel_sol.subs(y,-omegaxi).subs(alpha,a).subs(nu,Abs(l)).subs(sigma,0)\
.subs(Lj,Lk0).subs(j,k).subs(Li,Ln).subs(i,n)

rhs_simpl

                                                                              
                                                                              
                                                   ⎛   2⎞                ⎛   2
    k + n       │l│ + 1/2      -│l│ - 1       k - n⎜ωₓᵢ ⎟    -k + n + │l│⎜ωₓᵢ 
(-1)     ⋅(-ωₓᵢ)         ⋅(2⋅a)        ⋅L_k__0     ⎜────⎟⋅L_n            ⎜────
                                                   ⎝4⋅a ⎠                ⎝4⋅a 

       2 
   -ωₓᵢ  
⎞  ──────
⎟   4⋅a  
⎟⋅ℯ      
⎠        

### 1.8) Putting the two parts together

In [30]:
rhs_fact*rhs_simpl

                                                                              
                                                                              
                                                                              
      k + n  -│l′│ - 1  -l + l′    │l│  -n′ - │l′│ - 1               │l│ + 1/2
2⋅(-1)     ⋅2         ⋅ⅈ       ⋅a⋅a   ⋅b              ⋅g_k⋅κ⋅μ⋅(-ωₓᵢ)         
                                                                              
──────────────────────────────────────────────────────────────────────────────
                                                                              

                                                                              
                                                                              
                           │l′│                       ⎛   2⎞                ⎛ 
      -│l│ - 1 ⎛-c_n′⋅ωₓᵢ ⎞             n′       k - n⎜ωₓᵢ ⎟    -k + n + │l│⎜ω
⋅(2⋅a)        ⋅⎜──────────⎟    ⋅(-a + b)  ⋅L_k__0  

In [31]:
simplify(rhs_fact*rhs_simpl)

                                                                              
                                                                              
                                                                              
                                                                              
    k + n  -l + l′  │l│ + 1  -n′ - │l′│ - 1               │l│ + 1/2      -│l│ 
(-1)     ⋅ⅈ       ⋅a       ⋅b              ⋅g_k⋅κ⋅μ⋅(-ωₓᵢ)         ⋅(2⋅a)     
                                                                              
──────────────────────────────────────────────────────────────────────────────
                                                                              

                                  │l′│                                        
    ⎛              ⎛        2  ⎞ ⎞                                            
    ⎜              ⎜   a⋅ωₓᵢ   ⎟ ⎟                                            
    ⎜-c_n′⋅ωₓᵢ⋅L_n′⎜───────────⎟ ⎟                 

With the sums in front

In [32]:
Sum(rhs_fact*rhs_simpl,(lprime,-oo,oo),(nprime,0,oo),(k,0,n0))

  n₀     ∞       ∞                                                            
______ ______  ______                                                         
╲      ╲       ╲                                                              
 ╲      ╲       ╲                                                             
  ╲      ╲       ╲                                                            
   ╲      ╲       ╲                                                           
    ╲      ╲       ╲        k + n  -│l′│ - 1  -l + l′    │l│  -n′ - │l′│ - 1  
    ╱      ╱       ╱  2⋅(-1)     ⋅2         ⋅ⅈ       ⋅a⋅a   ⋅b              ⋅g
   ╱      ╱       ╱                                                           
  ╱      ╱       ╱    ────────────────────────────────────────────────────────
 ╱      ╱       ╱                                                             
╱      ╱       ╱                                                              
‾‾‾‾‾‾ ‾‾‾‾‾‾  ‾‾‾‾‾‾                               

In [33]:
import mpmath

## 2) RHS Impedance part

### 2.1) Expression from DELPHI-expanded.pdf

Expression as in DELPHI-expanded, slide 34 (without $ - \kappa g_0$)

In [45]:
tauprime

τ′

In [47]:
Jl, Jlprime = symbols("J_l J_lprime",cls=Function)
Zy = symbols("Z_y",cls=Function)
p, omegap = symbols("p omega_p")

In [48]:
impsum = Sum(Zy(omegap)*Jl((omegaxi-omegap)*tau)*Jlprime((omegaxi-omegap)*tauprime),(p,-oo,oo))
impsum

  ∞                                                      
 ___                                                     
 ╲                                                       
  ╲    J_l(τ⋅(-ω_p + ωₓᵢ))⋅J_l′(τ′⋅(-ω_p + ωₓᵢ))⋅Z_y(ω_p)
  ╱                                                      
 ╱                                                       
 ‾‾‾                                                     
p = -∞                                                   

In [49]:
expr=Integral(I**(lprime-l) * tauprime * Rlprime_tauprime* impsum,(tauprime, 0, oo))

expr=Sum(expr,(lprime, -oo, oo))
expr=simplify(expand_power_base(expr))

expr

    ∞                                                                         
_________                                                                     
╲                                                                             
 ╲        ∞                                                                   
  ╲       ⌠                                                                   
   ╲      ⎮                                                                   
    ╲     ⎮             ⎛  ∞                                                  
     ╲    ⎮             ⎜ ___                                                 
      ╲   ⎮             ⎜ ╲                                                   
       ╲  ⎮  -l + l′    ⎜  ╲    J_l(τ⋅(-ω_p + ωₓᵢ))⋅J_l′(τ′⋅(-ω_p + ωₓᵢ))⋅Z_y(
       ╱  ⎮ ⅈ       ⋅τ′⋅⎜  ╱                                                  
      ╱   ⎮             ⎜ ╱                                                   
     ╱    ⎮             ⎜ ‾‾‾                       

!!! The sums over $p$, $l'$ and $n'$ are not written in front !!!

In [64]:
newexpr=expr.args[0].args[0].args[0]*\
expr.args[0].args[0].args[1]*\
expr.args[0].args[0].args[2].args[0]*\
expr.args[0].args[0].args[3].args[0]

newexpr

                                                                              
 -l + l′     │l′│    -│l′│      │l′│                                          
ⅈ       ⋅c_n′    ⋅τ_b     ⋅τ′⋅τ′    ⋅J_l(τ⋅(-ω_p + ωₓᵢ))⋅J_l′(τ′⋅(-ω_p + ωₓᵢ))

                                2
     │l′│⎛    2⎞           -b⋅τ′ 
⋅L_n′    ⎝a⋅τ′ ⎠⋅Z_y(ω_p)⋅ℯ      

In [65]:
newexpr.args

⎛                                                                             
⎜     -l + l′      │l′│     -│l′│    │l′│      │l′│⎛    2⎞                    
⎝τ′, ⅈ       , c_n′    , τ_b     , τ′    , L_n′    ⎝a⋅τ′ ⎠, J_l(τ⋅(-ω_p + ωₓᵢ)

                                          2⎞
                                     -b⋅τ′ ⎟
), J_l′(τ′⋅(-ω_p + ωₓᵢ)), Z_y(ω_p), ℯ      ⎠

In [66]:
fact = np.prod([newexpr.args[ii] for ii in [1,2,3,6,8]])
fact

 -l + l′     │l′│    -│l′│                             
ⅈ       ⋅c_n′    ⋅τ_b     ⋅J_l(τ⋅(-ω_p + ωₓᵢ))⋅Z_y(ω_p)

In [67]:
newexpr=fact*simplify(Integral(newexpr/fact,(tauprime,0,oo)))
newexpr

                                                        ∞                     
                                                        ⌠                     
                                                        ⎮                     
 -l + l′     │l′│    -│l′│                              ⎮   │l′│ + 1          
ⅈ       ⋅c_n′    ⋅τ_b     ⋅J_l(τ⋅(-ω_p + ωₓᵢ))⋅Z_y(ω_p)⋅⎮ τ′        ⋅J_l′(τ′⋅(
                                                        ⌡                     
                                                        0                     

                                          
                                          
                                   2      
                 │l′│⎛    2⎞  -b⋅τ′       
-ω_p + ωₓᵢ))⋅L_n′    ⎝a⋅τ′ ⎠⋅ℯ       d(τ′)
                                          
                                          

### 2.2) Laguerre integral from Chao

Integral as from Chao p.316, table 6.3, formula 3

In [68]:
eta, alpha, y = symbols('eta, alpha, y')

laguerre_lhs=Integral(x**(lprime+1) *exp(-eta*x**2) *Jlprime(x*y)* Lnprime(alpha * x**2)**lprime,(x, 0, oo))

laguerre_lhs

∞                                         
⌠                                         
⎮                                     2   
⎮  l′ + 1               l′⎛   2⎞  -η⋅x    
⎮ x      ⋅J_l′(x⋅y)⋅L_n′  ⎝α⋅x ⎠⋅ℯ      dx
⌡                                         
0                                         

In [74]:
temp=laguerre_lhs.subs(alpha,a).subs(lprime,abs(lprime)).subs(y,omegaxi-omegap).subs(eta,b)
laguerre1_lhs=temp.transform(x,tauprime)

laguerre1_lhs

∞                                                               
⌠                                                               
⎮                                                        2      
⎮   │l′│ + 1                           │l′│⎛    2⎞  -b⋅τ′       
⎮ τ′        ⋅J_l′(τ′⋅(-ω_p + ωₓᵢ))⋅L_n′    ⎝a⋅τ′ ⎠⋅ℯ       d(τ′)
⌡                                                               
0                                                               

Solution

In [75]:
laguerre_rhs=(eta- alpha)**nprime/(2**(lprime+1)*eta**(lprime+nprime+1))*y**lprime*exp(-y**2/(4*eta))*\
Lnprime(alpha*y**2/(4*eta*(alpha-eta)))**lprime
laguerre_rhs

                                                             2 
                                                           -y  
                                            ⎛       2   ⎞  ────
 -l′ - 1  -l′ - n′ - 1  l′         n′     l′⎜    α⋅y    ⎟  4⋅η 
2       ⋅η            ⋅y  ⋅(-α + η)  ⋅L_n′  ⎜───────────⎟⋅ℯ    
                                            ⎝4⋅η⋅(α - η)⎠      

In [76]:
laguerre1_rhs=laguerre_rhs.subs(alpha,a).subs(lprime,abs(lprime)).subs(y,omegaxi-omegap).subs(eta,b)

laguerre1_rhs

                                                                              
                                                                              
                                                               ⎛              
 -│l′│ - 1  -n′ - │l′│ - 1         n′             │l′│     │l′│⎜a⋅(-ω_p + ωₓᵢ)
2         ⋅b              ⋅(-a + b)  ⋅(-ω_p + ωₓᵢ)    ⋅L_n′    ⎜──────────────
                                                               ⎝  4⋅b⋅(a - b) 

                 2 
    -(-ω_p + ωₓᵢ)  
2⎞  ───────────────
 ⎟        4⋅b      
─⎟⋅ℯ               
 ⎠                 

### 2.3)  Substituting Laguerre 1 in DELPHI's expression

In [93]:
newexpr0=Sum(simplify(newexpr.subs(laguerre1_lhs,laguerre1_rhs)),(p,-oo,oo),(lprime, -oo, oo),(nprime, 0, oo))
newexpr0

  ∞       ∞      ∞                                                            
______  ______ ______                                                         
╲       ╲      ╲                                                              
 ╲       ╲      ╲                                                             
  ╲       ╲      ╲                                        ⎛                   
   ╲       ╲      ╲                                       ⎜                   
    ╲       ╲      ╲                                      ⎜c_n′⋅(-ω_p + ωₓᵢ)⋅L
    ╱       ╱      ╱   -│l′│ - 1  -l + l′  -n′ - │l′│ - 1 ⎜                   
   ╱       ╱      ╱   2         ⋅ⅈ       ⋅b              ⋅⎜───────────────────
  ╱       ╱      ╱                                        ⎝                 τ_
 ╱       ╱      ╱                                                             
╱       ╱      ╱                                                              
‾‾‾‾‾‾  ‾‾‾‾‾‾ ‾‾‾‾‾‾                               

In [94]:
newexpr0.args[0].args

⎛                                                                             
⎜                                       ⎛                      ⎛             2
⎜                                       ⎜                      ⎜a⋅(ω_p - ωₓᵢ) 
⎜                                       ⎜c_n′⋅(-ω_p + ωₓᵢ)⋅L_n′⎜──────────────
⎜ -│l′│ - 1   -l + l′   -n′ - │l′│ - 1  ⎜                      ⎝ 4⋅b⋅(a - b)  
⎜2         , ⅈ       , b              , ⎜─────────────────────────────────────
⎝                                       ⎝                 τ_b                 

  │l′│                                                            ⎞
⎞⎞                                                              2 ⎟
⎟⎟                                                  -(ω_p - ωₓᵢ)  ⎟
⎟⎟                                                  ──────────────⎟
⎠⎟              n′                                       4⋅b      ⎟
─⎟    , (-a + b)  , J_l(-τ⋅(ω_p - ωₓᵢ)), Z_y(ω_p), ℯ              ⎟
 ⎠                                    

In [95]:
np.prod([newexpr0.args[0].args[ii] for ii in [0,1,2,4,5,6,7]])

                                                                              
                                                                             -
                                                                             ─
 -│l′│ - 1  -l + l′  -n′ - │l′│ - 1         n′                                
2         ⋅ⅈ       ⋅b              ⋅(-a + b)  ⋅J_l(-τ⋅(ω_p - ωₓᵢ))⋅Z_y(ω_p)⋅ℯ 

           2 
(ω_p - ωₓᵢ)  
─────────────
    4⋅b      
             

In [91]:
(np.prod([newexpr0.args[0].args[3].args[0].args[ii] for ii in [0,1,2]]))**\
(newexpr0.args[0].args[3].args[1])*\
(newexpr0.args[0].args[3].args[0].args[3])**(newexpr0.args[0].args[3].args[1])

                   │l′│         ⎛             2⎞
⎛c_n′⋅(-ω_p + ωₓᵢ)⎞         │l′│⎜a⋅(ω_p - ωₓᵢ) ⎟
⎜─────────────────⎟    ⋅L_n′    ⎜──────────────⎟
⎝       τ_b       ⎠             ⎝ 4⋅b⋅(a - b)  ⎠

In [96]:
newexpr0 = np.prod([newexpr0.args[0].args[ii] for ii in [0,1,2,4,5,6,7]])*\
(np.prod([newexpr0.args[0].args[3].args[0].args[ii] for ii in [0,1,2]]))**\
(newexpr0.args[0].args[3].args[1])*\
(newexpr0.args[0].args[3].args[0].args[3])**(newexpr0.args[0].args[3].args[1])
newexpr0

                                                                              
                                                                              
                                                       │l′│                   
 -│l′│ - 1  -l + l′  -n′ - │l′│ - 1 ⎛c_n′⋅(-ω_p + ωₓᵢ)⎞             n′        
2         ⋅ⅈ       ⋅b              ⋅⎜─────────────────⎟    ⋅(-a + b)  ⋅J_l(-τ⋅
                                    ⎝       τ_b       ⎠                       

                                                            2 
                                                -(ω_p - ωₓᵢ)  
                     ⎛             2⎞           ──────────────
                 │l′│⎜a⋅(ω_p - ωₓᵢ) ⎟                4⋅b      
(ω_p - ωₓᵢ))⋅L_n′    ⎜──────────────⎟⋅Z_y(ω_p)⋅ℯ              
                     ⎝ 4⋅b⋅(a - b)  ⎠                         

In [98]:
newexpr0 = Sum(newexpr0,(p,-oo,oo), (lprime, -oo, oo), (nprime, 0, oo))
newexpr0

  ∞       ∞      ∞                                                            
_____   _____  _____                                                          
╲       ╲      ╲                                                              
 ╲       ╲      ╲                                                             
  ╲       ╲      ╲                                                            
   ╲       ╲      ╲                                                          │
   ╱       ╱      ╱    -│l′│ - 1  -l + l′  -n′ - │l′│ - 1 ⎛c_n′⋅(-ω_p + ωₓᵢ)⎞ 
  ╱       ╱      ╱    2         ⋅ⅈ       ⋅b              ⋅⎜─────────────────⎟ 
 ╱       ╱      ╱                                         ⎝       τ_b       ⎠ 
╱       ╱      ╱                                                              
‾‾‾‾‾   ‾‾‾‾‾  ‾‾‾‾‾                                                          
n′ = 0 l′ = -∞ p = -∞                                                         

                                                   

### 2.4) Product with $- \kappa g_0$

In [100]:
kappa = symbols('kappa')

newexpr2 = -kappa * g0* newexpr0

newexpr2

                                     ∞       ∞      ∞                         
                                   _____   _____  _____                       
          ⎛  n₀                  ⎞ ╲       ╲      ╲                           
          ⎜ ___                  ⎟  ╲       ╲      ╲                          
        2 ⎜ ╲                    ⎟   ╲       ╲      ╲                         
    -b⋅τ  ⎜  ╲             ⎛   2⎞⎟    ╲       ╲      ╲                        
-κ⋅ℯ     ⋅⎜  ╱   g_k⋅L_k__0⎝a⋅τ ⎠⎟⋅   ╱       ╱      ╱    -│l′│ - 1  -l + l′  
          ⎜ ╱                    ⎟   ╱       ╱      ╱    2         ⋅ⅈ       ⋅b
          ⎜ ‾‾‾                  ⎟  ╱       ╱      ╱                          
          ⎝k = 0                 ⎠ ╱       ╱      ╱                           
                                   ‾‾‾‾‾   ‾‾‾‾‾  ‾‾‾‾‾                       
                                   n′ = 0 l′ = -∞ p = -∞                      

                                                   

In [109]:
newexpr3 = \
newexpr2.args[0]*\
newexpr2.args[1]*\
newexpr2.args[2].args[0]*\
newexpr2.args[3].args[0]*\
newexpr2.args[4]

                                                                              
                                                                              
                                                              │l′│            
  -│l′│ - 1  -l + l′  -n′ - │l′│ - 1       ⎛c_n′⋅(-ω_p + ωₓᵢ)⎞             n′ 
-2         ⋅ⅈ       ⋅b              ⋅g_k⋅κ⋅⎜─────────────────⎟    ⋅(-a + b)  ⋅
                                           ⎝       τ_b       ⎠                

                                                                              
                                                                    -(ω_p - ωₓ
                                         ⎛             2⎞           ──────────
                          ⎛   2⎞     │l′│⎜a⋅(ω_p - ωₓᵢ) ⎟                4⋅b  
J_l(-τ⋅(ω_p - ωₓᵢ))⋅L_k__0⎝a⋅τ ⎠⋅L_n′    ⎜──────────────⎟⋅Z_y(ω_p)⋅ℯ          
                                         ⎝ 4⋅b⋅(a - b)  ⎠                     

  2        
ᵢ)         
────      2
      -b⋅τ 
  

The product, once the sums have been put in front

!!! The sums are not written !!! They are on $-\infty \leq p \leq \infty$

$0 \leq n' \leq \infty$

$-\infty \leq l' \leq \infty$

$0 \leq k \leq n_0$

In [110]:
newexpr3

                                                                              
                                                                              
                                                              │l′│            
  -│l′│ - 1  -l + l′  -n′ - │l′│ - 1       ⎛c_n′⋅(-ω_p + ωₓᵢ)⎞             n′ 
-2         ⋅ⅈ       ⋅b              ⋅g_k⋅κ⋅⎜─────────────────⎟    ⋅(-a + b)  ⋅
                                           ⎝       τ_b       ⎠                

                                                                              
                                                                    -(ω_p - ωₓ
                                         ⎛             2⎞           ──────────
                          ⎛   2⎞     │l′│⎜a⋅(ω_p - ωₓᵢ) ⎟                4⋅b  
J_l(-τ⋅(ω_p - ωₓᵢ))⋅L_k__0⎝a⋅τ ⎠⋅L_n′    ⎜──────────────⎟⋅Z_y(ω_p)⋅ℯ          
                                         ⎝ 4⋅b⋅(a - b)  ⎠                     

  2        
ᵢ)         
────      2
      -b⋅τ 
  

### 2.5) Multiplying by the weight function and integrating in $\tau$

The weight function is necessary for the orthogonality on the LHS

In [111]:
omega, qy0, omega0, omegas = symbols('Omega Q_y_0 omega_0 omega_s')

weight = Ln(a*tau**2)**abs(l)*exp((-a+b)*tau**2)*tau**abs(l)*2*a*tau*a**(abs(l))
weight

                               2         
     │l│    │l│    │l│⎛   2⎞  τ ⋅(-a + b)
2⋅a⋅a   ⋅τ⋅τ   ⋅L_n   ⎝a⋅τ ⎠⋅ℯ           

In [112]:
rhs_out = newexpr3*weight
rhs_out

                                                                              
                                                                              
                                                                              
    -│l′│ - 1  -l + l′    │l│  -n′ - │l′│ - 1          │l│ ⎛c_n′⋅(-ω_p + ωₓᵢ)⎞
-2⋅2         ⋅ⅈ       ⋅a⋅a   ⋅b              ⋅g_k⋅κ⋅τ⋅τ   ⋅⎜─────────────────⎟
                                                           ⎝       τ_b       ⎠

                                                                              
                                                                              
│l′│                                                                  ⎛       
             n′                           ⎛   2⎞    │l│⎛   2⎞     │l′│⎜a⋅(ω_p 
    ⋅(-a + b)  ⋅J_l(-τ⋅(ω_p - ωₓᵢ))⋅L_k__0⎝a⋅τ ⎠⋅L_n   ⎝a⋅τ ⎠⋅L_n′    ⎜───────
                                                                      ⎝ 4⋅b⋅(a

                               2                  

In [114]:
args=np.array(rhs_out.args)

rhs_fact=np.prod([args[ii] for ii in [0,1,2,3,5,6,7,8,10,11,13,16,19]])

rhs_fact

                                                                              
                                                                              
                                                                       │l′│   
    -│l′│ - 1  -l + l′    │l│  -n′ - │l′│ - 1       ⎛c_n′⋅(-ω_p + ωₓᵢ)⎞       
-2⋅2         ⋅ⅈ       ⋅a⋅a   ⋅b              ⋅g_k⋅κ⋅⎜─────────────────⎟    ⋅(-
                                                    ⎝       τ_b       ⎠       

                                                        2 
                                            -(ω_p - ωₓᵢ)  
                 ⎛             2⎞           ──────────────
      n′     │l′│⎜a⋅(ω_p - ωₓᵢ) ⎟                4⋅b      
a + b)  ⋅L_n′    ⎜──────────────⎟⋅Z_y(ω_p)⋅ℯ              
                 ⎝ 4⋅b⋅(a - b)  ⎠                         

And integrating in $\tau$

In [115]:
#rhs_fact = 2*2**(-Abs(lprime) - 1)*I**(-l + lprime)*a*a**Abs(l)*b**(-nprime - Abs(lprime) - 1)*\
#gk*kappa*(-cnprime*omegaxi/taub)**Abs(lprime)*(-a + b)**nprime*\
#Lnprime(a*omegaxi**2/(4*b*(a - b)))**Abs(lprime)*exp(-omegaxi**2/(4*b))

rhs_int = np.prod([args[ii] for ii in [4,9,12,14,15,17,18]])
rhs_int = simplify(rhs_int)
rhs_int = Integral(rhs_int,(tau,0,oo))

rhs_int

∞                                                                 
⌠                                                                 
⎮                                                             2   
⎮  │l│ + 1                           ⎛   2⎞    │l│⎛   2⎞  -a⋅τ    
⎮ τ       ⋅J_l(τ⋅(-ω_p + ωₓᵢ))⋅L_k__0⎝a⋅τ ⎠⋅L_n   ⎝a⋅τ ⎠⋅ℯ      dτ
⌡                                                                 
0                                                                 

In [116]:
rhs_fact*rhs_int

                                                                              
                                                                              
                                                                       │l′│   
    -│l′│ - 1  -l + l′    │l│  -n′ - │l′│ - 1       ⎛c_n′⋅(-ω_p + ωₓᵢ)⎞       
-2⋅2         ⋅ⅈ       ⋅a⋅a   ⋅b              ⋅g_k⋅κ⋅⎜─────────────────⎟    ⋅(-
                                                    ⎝       τ_b       ⎠       
                                                                              

                                                        2  ∞                  
                                            -(ω_p - ωₓᵢ)   ⌠                  
                 ⎛             2⎞           ────────────── ⎮                  
      n′     │l′│⎜a⋅(ω_p - ωₓᵢ) ⎟                4⋅b       ⎮  │l│ + 1         
a + b)  ⋅L_n′    ⎜──────────────⎟⋅Z_y(ω_p)⋅ℯ              ⋅⎮ τ       ⋅J_l(τ⋅(-
                 ⎝ 4⋅b⋅(a - b)  ⎠                  

### 2.6) Formula form Erdelyi, vol 2, p43, formula 8 

In [117]:
x, y, alpha, nu, sigma = symbols("x y alpha nu sigma")
i, j = symbols("i j")
Li, Lj = symbols("L_i L_j", cls=Function)
Jnu = symbols("J_nu", cls=Function)

In [118]:
erdel_int = Integral(x**(nu+S(1)/2)*exp(-alpha*x**2)*Li(alpha*x**2)**(nu-sigma)*\
                      Lj(alpha*x**2)**sigma*Jnu(x*y)*(x*y)**(S(1)/2),(x,0,oo))

erdel_int
simplify(erdel_int)

∞                                                             
⌠                                                             
⎮                                                         2   
⎮  ν + 1/2   _____            ν - σ⎛   2⎞    σ⎛   2⎞  -α⋅x    
⎮ x       ⋅╲╱ x⋅y ⋅J_ν(x⋅y)⋅Lᵢ     ⎝α⋅x ⎠⋅L_j ⎝α⋅x ⎠⋅ℯ      dx
⌡                                                             
0                                                             

In [119]:
erdel_sol = (-1)**(i+j)*(2*alpha)**(-nu-1)*y**(nu+S(1)/2)*exp(-y**2/(4*alpha))*\
Lj(y**2/(4*alpha))**(sigma-i+j)*Li(y**2/(4*alpha))**(nu-sigma+i-j)

erdel_sol

                                                                          2 
                                                                        -y  
                                              ⎛  2⎞              ⎛  2⎞  ────
    i + j  ν + 1/2      -ν - 1   i - j + ν - σ⎜ y ⎟    -i + j + σ⎜ y ⎟  4⋅α 
(-1)     ⋅y       ⋅(2⋅α)      ⋅Lᵢ             ⎜───⎟⋅L_j          ⎜───⎟⋅ℯ    
                                              ⎝4⋅α⎠              ⎝4⋅α⎠      

### 2.7) Using the formula

In [124]:
rhs_int

∞                                                                 
⌠                                                                 
⎮                                                             2   
⎮  │l│ + 1                           ⎛   2⎞    │l│⎛   2⎞  -a⋅τ    
⎮ τ       ⋅J_l(τ⋅(-ω_p + ωₓᵢ))⋅L_k__0⎝a⋅τ ⎠⋅L_n   ⎝a⋅τ ⎠⋅ℯ      dτ
⌡                                                                 
0                                                                 

- $\sigma$ is 0
- $\nu$ is abs(l)
- i is n
- j is k
- y is $-\omega_p$ + $\omega_{xi}$
- $\alpha$ is a

In [130]:
rhs_simpl = simplify((omegaxi-omegap)**(-S(1)/2)*erdel_sol.subs(y,-omegap+omegaxi).subs(alpha,a).subs(nu,Abs(l)).subs(sigma,0)\
.subs(Lj,Lk0).subs(j,k).subs(Li,Ln).subs(i,n))

rhs_simpl

                                                                              
                                                                              
                                                   ⎛           2⎞             
    k + n      -│l│ - 1             │l│       k - n⎜(ω_p - ωₓᵢ) ⎟    -k + n + 
(-1)     ⋅(2⋅a)        ⋅(-ω_p + ωₓᵢ)   ⋅L_k__0     ⎜────────────⎟⋅L_n         
                                                   ⎝    4⋅a     ⎠             

                               2 
                   -(ω_p - ωₓᵢ)  
   ⎛           2⎞  ──────────────
│l│⎜(ω_p - ωₓᵢ) ⎟       4⋅a      
   ⎜────────────⎟⋅ℯ              
   ⎝    4⋅a     ⎠                

### 2.8) Putting the two parts together

In [131]:
rhs_fact*rhs_simpl

                                                                              
                                                                              
                                                                              
       k + n  -│l′│ - 1  -l + l′    │l│  -n′ - │l′│ - 1            -│l│ - 1 ⎛c
-2⋅(-1)     ⋅2         ⋅ⅈ       ⋅a⋅a   ⋅b              ⋅g_k⋅κ⋅(2⋅a)        ⋅⎜─
                                                                            ⎝ 

                                                                              
                                                                              
                 │l′│                                       ⎛           2⎞    
_n′⋅(-ω_p + ωₓᵢ)⎞             n′             │l│       k - n⎜(ω_p - ωₓᵢ) ⎟    
────────────────⎟    ⋅(-a + b)  ⋅(-ω_p + ωₓᵢ)   ⋅L_k__0     ⎜────────────⎟⋅L_n
      τ_b       ⎠                                           ⎝    4⋅a     ⎠    

                                                  

In [133]:
Sum(rhs_fact*rhs_simpl,(p,-oo,oo),(lprime,-oo,oo),(nprime,0,oo),(k,0,n0))

  n₀    ∞       ∞      ∞                                                      
_____ _____   _____  _____                                                    
╲     ╲       ╲      ╲                                                        
 ╲     ╲       ╲      ╲                                                       
  ╲     ╲       ╲      ╲                                                      
   ╲     ╲       ╲      ╲                                                     
   ╱     ╱       ╱      ╱          k + n  -│l′│ - 1  -l + l′    │l│  -n′ - │l′
  ╱     ╱       ╱      ╱    -2⋅(-1)     ⋅2         ⋅ⅈ       ⋅a⋅a   ⋅b         
 ╱     ╱       ╱      ╱                                                       
╱     ╱       ╱      ╱                                                        
‾‾‾‾‾ ‾‾‾‾‾   ‾‾‾‾‾  ‾‾‾‾‾                                                    
k = 0 n′ = 0 l′ = -∞ p = -∞                                                   

                                                   

## 3) LHS

In [35]:
omega, qy0, omega0, omegas = symbols('Omega Q_y_0 omega_0 omega_s')
Lnprime = symbols("L_nprime",cls=Function)
lhs=(omega - qy0*omega0 - l*omegas)*Rl

weight = Ln(a*tau**2)**abs(l)*exp((-a+b)*tau**2)*tau**abs(l)*2*a*tau*a**(abs(l))
lhs*= weight
lhs

                                                                   ∞          
                                                                  ____        
                                                                  ╲           
                                                      2            ╲          
     │l│    │l│                           │l│⎛   2⎞  τ ⋅(-a + b)    ╲       │l
2⋅a⋅a   ⋅τ⋅τ   ⋅(Ω - Q_y_0⋅ω₀ - l⋅ω_s)⋅L_n   ⎝a⋅τ ⎠⋅ℯ           ⋅   ╱   c_n′  
                                                                   ╱          
                                                                  ╱           
                                                                  ‾‾‾‾        
                                                                 n′ = 0       

                               
                               
                               
       │l│                    2
│ ⎛ τ ⎞        │l│⎛   2⎞  -b⋅τ 
 ⋅⎜───⎟   ⋅L_n′   ⎝a⋅τ ⎠⋅ℯ     
  ⎝τ_b⎠          

In [36]:
lhs.args

⎛                                                           ∞                 
⎜                                                          ____               
⎜                                                          ╲                  
⎜                                                           ╲                 
⎜          │l│   │l│     │l│⎛   2⎞                           ╲       │l│ ⎛ τ ⎞
⎜2, a, τ, a   , τ   , L_n   ⎝a⋅τ ⎠, Ω - Q_y_0⋅ω₀ - l⋅ω_s,    ╱   c_n′   ⋅⎜───⎟
⎜                                                           ╱            ⎝τ_b⎠
⎜                                                          ╱                  
⎜                                                          ‾‾‾‾               
⎝                                                         n′ = 0              

                                      ⎞
                                      ⎟
                                      ⎟
│l│                    2    2         ⎟
        │l│⎛   2⎞  -b⋅τ    τ ⋅(-a + b)⎟
   ⋅L_n′ 

In [37]:
args=np.array(lhs.args)
out=np.prod([args[ii] for ii in [0,1,2,3,4,5,6,8]])
out*=args[7].args[0]
out=simplify(out)
out=expand_power_base(out,tau,force=True)
out

                                                                              
   │l│ + 1     │l│  2⋅│l│ + 1    -│l│                           │l│⎛   2⎞     
2⋅a       ⋅c_n′   ⋅τ         ⋅τ_b    ⋅(Ω - Q_y_0⋅ω₀ - l⋅ω_s)⋅L_n   ⎝a⋅τ ⎠⋅L_n′

               2
│l│⎛   2⎞  -a⋅τ 
   ⎝a⋅τ ⎠⋅ℯ     

In [38]:
# getting the series out

# integrating in tau

fact = cnprime**abs(l)*(omega - qy0*omega0 - l*omegas)*taub**(-abs(l))

temp=fact*Integral(out*1/fact,(tau,0,oo))
temp=expand_power_exp(temp)
temp
# print(temp)

                                       ∞                                      
                                       ⌠                                      
                                       ⎮                                      
    │l│    -│l│                        ⎮      │l│    2⋅│l│    │l│⎛   2⎞     │l
c_n′   ⋅τ_b    ⋅(Ω - Q_y_0⋅ω₀ - l⋅ω_s)⋅⎮ 2⋅a⋅a   ⋅τ⋅τ     ⋅L_n   ⎝a⋅τ ⎠⋅L_n′  
                                       ⌡                                      
                                       0                                      

                 
                 
             2   
│⎛   2⎞  -a⋅τ    
 ⎝a⋅τ ⎠⋅ℯ      dτ
                 
                 

#### 3.1) Laguerre ortogonality


In [39]:
# a=symbols('a',positive=True)
lag2=Integral(exp(-x)*x**l*Ln(x)**l*Lnprime(x)**l,(x,0,oo))
lag2=lag2.subs(l,abs(l))
lag2=lag2.transform(x,(a*tau**2,tau))
lag2=expand_power_base(lag2)
lag2

∞                                                       
⌠                                                       
⎮                                                   2   
⎮      │l│    2⋅│l│    │l│⎛   2⎞     │l│⎛   2⎞  -a⋅τ    
⎮ 2⋅a⋅a   ⋅τ⋅τ     ⋅L_n   ⎝a⋅τ ⎠⋅L_n′   ⎝a⋅τ ⎠⋅ℯ      dτ
⌡                                                       
0                                                       

#### Solution

In [40]:
nprime=symbols('nprime',positive=True)
sol2=factorial(abs(l)+nprime)/factorial(nprime)*KroneckerDelta(n,nprime)
sol2

δ    ⋅(n′ + │l│)!
 n,n′            
─────────────────
       n′!       

In [41]:
temp.args[3], lag2

⎛∞                                                         ∞                  
⎜⌠                                                         ⌠                  
⎜⎮                                                   2     ⎮                  
⎜⎮      │l│    2⋅│l│    │l│⎛   2⎞     │l│⎛   2⎞  -a⋅τ      ⎮      │l│    2⋅│l│
⎜⎮ 2⋅a⋅a   ⋅τ⋅τ     ⋅L_n   ⎝a⋅τ ⎠⋅L_n′   ⎝a⋅τ ⎠⋅ℯ      dτ, ⎮ 2⋅a⋅a   ⋅τ⋅τ     
⎜⌡                                                         ⌡                  
⎝0                                                         0                  

                                     ⎞
                                     ⎟
                                 2   ⎟
    │l│⎛   2⎞     │l│⎛   2⎞  -a⋅τ    ⎟
⋅L_n   ⎝a⋅τ ⎠⋅L_n′   ⎝a⋅τ ⎠⋅ℯ      dτ⎟
                                     ⎟
                                     ⎠

In [42]:
temp=temp.subs(lag2,sol2)
temp

# putting back the sum in nprime
lhs=Sum(temp,(nprime,0,oo))
lhs

  ∞                                                           
_____                                                         
╲                                                             
 ╲        │l│    -│l│                                         
  ╲   c_n′   ⋅τ_b    ⋅(Ω - Q_y_0⋅ω₀ - l⋅ω_s)⋅δ    ⋅(n′ + │l│)!
   ╲                                          n,n′            
   ╱  ────────────────────────────────────────────────────────
  ╱                             n′!                           
 ╱                                                            
╱                                                             
‾‾‾‾‾                                                         
False                                                         

### LHS solved

In [43]:
lhs.args[0].subs(KroneckerDelta(n,nprime),1).subs(nprime,n).subs(cnprime,cn)

   │l│    -│l│                                  
c_n   ⋅τ_b    ⋅(Ω - Q_y_0⋅ω₀ - l⋅ω_s)⋅(n + │l│)!
────────────────────────────────────────────────
                       n!                       